## Introduction
This notebook is a more serious attempt at correcting for variation in powder peak postions through a combination of a fit to the slow-period variations due to variation the sample surface location, and fast shot-to-shot fluctuations due to XFEL energy jitter.

## Background subtraction

In order to properly measure peak position shifts we need to do background subtraction on every event for the low signal-to-background peaks, or else we need to infer the variation in low-signal peaks based on that in high-signal peaks.

In [1]:
!hostname

psanaphi102


In [1]:
!mv cache old_cache2

In [2]:
!rm -r cache/*

In [1]:
#%matplotlib inline
%matplotlib nbagg
import config
config.autobatch = False
config.multiprocess = True
config.autompi = False
#config.testing = True

#config.peak_width = 0.25 # Change from default value of 1.5.
from dataccess import nbfunctions
from dataccess.nbfunctions import *
from dataccess import utils, query, mec, datashow
from dataccess.summarymetrics import plt, histogram, scatter
import config

from scipy.stats import pearsonr
from scipy import stats
from dataccess.peakfinder import peakfilter_frame
#from dataccess.lk20 import ith_peak_cms_bgsubbed
from scipy.ndimage.filters import gaussian_filter as gfilt
from scipy.interpolate import interp1d
from dataccess.xtcav import autocorrelation
from dataccess import mec
%pdb

Automatic pdb calling has been turned ON


In [2]:
MgO_ref_short_full = [query.DataSet([906], label = 'MgO.ref.1C.short.2um')] + [get_query_dataset('material MgO runs 870 961 transmission 1. focal_size %d' % spotsize,
                                'MgO.ref.1C.short2.%sum' % spotsize, print_enable = False)
                                for spotsize in [4, 8, 18, 38, 48]] + [query.DataSet([922], label = 'MgO.ref.1C.short.58um')]
MgO_ref_short_clean = [get_query_dataset('material MgO runs 870 961 transmission 1. focal_size %d' % spotsize,
                                'MgO.ref.1C.short.%sum' % spotsize, print_enable = False)
                                for spotsize in [4, 8, 18, 38, 48]] + [query.DataSet([922], label = 'MgO.ref.1C.short.58um')]
datasets = MgO_ref_short_full
ds = datasets[1]

In [3]:
def ith_peak_cm0(i):
    def powder_pattern_cm(imarr = None, **kwargs):
        import numpy as np
        from dataccess.peakfinder import peakfilter_frame
        from dataccess import xrd
        dss = xrd.Pattern.from_dataset(peakfilter_frame(imarr, detid = 'quad2'), 'quad2', ['MgO'], label  = 'test')
        return dss.centers_of_mass()[i]
    return powder_pattern_cm

In [4]:
def cm_variation_scatter_plot0(peak_index):
    trace = ds.evaluate('quad2', frame_processor = ith_peak_cm0(peak_index), event_data_getter = utils.identity)
    event_numbers, cms = range(len(trace.flat_event_data())), trace.flat_event_data()
    smoothed_cm_interp = utils.extrap1d(interp1d(event_numbers, gfilt(cms, 20)))
    smoothed_cms = smoothed_cm_interp(event_numbers)
    cms_highpass = cms - smoothed_cms
    from dataccess.mec import si_imarr_cm_3
    energies = ds.evaluate('si', frame_processor = si_imarr_cm_3, event_data_getter = utils.identity)

    plt.scatter(event_numbers, cms, label = 'raw')
    plt.plot(np.array(event_numbers), smoothed_cms, label  ='interpolation, smoothed with sigma = 20')
    plt.show()
    


In [5]:
config.beam_intensity_diagnostic('906')

0.00070475531660561244

In [7]:
from dataccess import default_config

In [9]:
config.multiprocess

False

In [10]:
config.multiprocess

False

In [ ]:
config.multi

In [6]:
ds.evaluate('ipm2')

None
Waiting for engines...
<module 'ipyparallel.serialize' from '/reg/neh/home/ohoidn/venv_test/mecana2/lib/python2.7/site-packages/ipyparallel/serialize/__init__.pyc'>

restarting
Waiting for engines...
<module 'ipyparallel.serialize' from '/reg/neh/home/ohoidn/venv_test/mecana2/lib/python2.7/site-packages/ipyparallel/serialize/__init__.pyc'>

restarting


DataResultBase(mean=1.2447527304463002, event_data={911: {0: 1.2365911, 1: 1.3981842, 2: 1.2499428, 3: 1.3709468, 4: 1.3507286, 5: 1.3781948, 6: 1.1766995, 7: 1.1485466, 8: 1.0177767, 9: 1.1033798, 10: 1.1868467, 11: 0.86259252, 12: 1.2103456, 13: 1.1570916, 14: 1.2019532, 15: 1.2519264, 16: 1.1608301, 17: 1.249485, 18: 1.5078965, 19: 1.0493629, 20: 1.5404745, 21: 1.4529641, 22: 1.0646982, 23: 1.1036087, 24: 1.121767, 25: 1.1790646, 26: 1.1458763, 27: 1.2355993, 28: 0.99839783, 29: 1.0253299, 30: 1.4200809, 31: 0.96192873, 32: 1.3263142, 33: 1.1107042, 34: 1.1024643, 35: 0.98039216, 36: 0.62760359, 37: 1.1488518, 38: 1.2619212, 39: 0.9765774, 40: 1.0522622, 41: 1.2063019, 42: 0.90615702, 43: 1.2063019, 44: 1.6715496, 45: 0.71061265, 46: 1.3361562, 47: 0.94972152, 48: 1.1864653, 49: 1.4820325, 50: 1.4522774, 51: 1.3481346, 52: 0.98878461, 53: 0.97566187, 54: 1.1548791, 55: 0.83016711, 56: 1.4212253, 57: 1.5841917, 58: 0.74479288, 59: 1.5116349, 60: 1.175784, 61: 1.2172122, 62: 1.2697033

In [10]:
ds2 = query.DataSet([907], label = 'r907')
ds3= query.DataSet([908], label = 'rr908')

In [10]:
import psana

In [11]:
psana.DataSource

<function psana.datasource.DataSource>

In [16]:
r530 = query.DataSet([531], label = 'r531')
res530 = r530.evaluate('si')
plt.imshow(res530.mean)

An error has occured during the function import
Traceback (most recent call last):
  File "/reg/neh/home5/ohoidn/venv_test/mecana2/lib/python2.7/site-packages/ppworker.py", line 86, in run
    exec __fobj
  File "<string>", line 1, in <module>
ImportError: cannot import name result
A fatal error has occured during the function execution
Traceback (most recent call last):
  File "/reg/neh/home5/ohoidn/venv_test/mecana2/lib/python2.7/site-packages/ppworker.py", line 95, in run
    __f = locals()[__fname]
KeyError: 'mapfunc'
An error has occured during the function import
Traceback (most recent call last):
  File "/reg/neh/home5/ohoidn/venv_test/mecana2/lib/python2.7/site-packages/ppworker.py", line 86, in run
    exec __fobj
  File "<string>", line 1, in <module>
ImportError: cannot import name result
A fatal error has occured during the function execution
Traceback (most recent call last):
  File "/reg/neh/home5/ohoidn/venv_test/mecana2/lib/python2.7/site-packages/ppworker.py", line 95,

TypeError: zip argument #1 must support iteration

> /reg/neh/home5/ohoidn/venv_test/mecana2/lib/python2.7/site-packages/dataccess-1.0-py2.7.egg/dataccess/psget.py(527)multiprocess_func()
    525                 log(str(e))
    526                 gathered = map(mapfunc, range(1))
--> 527         signalsum_list, event_data_list, events_processed_list  = zip(*gathered)
    528         signalsum = reduce(lambda x, y: x + y, signalsum_list)
    529         events_processed = reduce(lambda x, y: x + y, events_processed_list)

ipdb> gathered
[None, None, None, None, None, None, None, None, None, None, None, None]
ipdb> c


In [11]:
config.multiprocess = False

In [5]:
cm_variation_scatter_plot0(1)
#cm_variation_scatter_plot(2)

In [29]:
np.mean(my_xrd.patterns[0].intensities)

18.586332466306338

In [8]:
np.mean(my_xrd.patterns[0].intensities)

18.586332466306338

In [35]:
pat = my_xrd.patterns[0]
pat2 = xrd.Pattern.from_dataset(pat.image, 'quad2', ['MgO'], label  = 'test')

In [40]:
np.mean(pat.image)

10.05193354679489

In [7]:
def bgfit_check(dataset):
    powder_pattern_cm, my_xrd = peak_cm(dataset)
    pat = my_xrd.patterns[0]
    fits = pat.fit_backgrounds()
    xfit = reduce(lambda x, y: np.concatenate((x, y)), [fit.xfit for fit in fits])
    yfit = reduce(lambda x, y: np.concatenate((x, y)), [fit.yfit for fit in fits])
    plt.scatter(xfit, yfit, label = '')
    plt.plot(pat.angles, pat.intensities)
    plt.show()

In [16]:
bgfit_check(ds)

In [29]:
cms, smoothed_cms, smoothed_cm_interp = cm_variation_scatter(ds, bgsub = True)

NameError: name 'cm_variation_scatter' is not defined

In [6]:
#correct_cms = nbfunctions.cm_variation_scatter(ds, bgsub = True, correct_cm = True)

In [5]:
def signal_plus_background_trace(ds):
    """
    Return peak integrals for all events in this dataset.
    """
    from dataccess.peakfinder import peakfilter_frame

    def one_event(imarr = None, **kwargs):
        cleaned_quad2 = peakfilter_frame(imarr, detid = 'quad2')
        # TODO is detid in the kwargs?
        pat = xrd.Pattern.from_dataset(cleaned_quad2, 'quad2', ['MgO'], label  = 'test')
        return np.array([peak.integrate(pat.angles, pat.intensities) for peak in pat.peaks.peaks])
        #return peak.integrate(angles, intensities, mode = 'integral')
    result = ds.evaluate('quad2', frame_processor = one_event,
                            event_data_getter = utils.identity)
    return result.flat_event_data()
    #angles, intensities = result.mean
    #pattern = xrd.Pattern(angles, intensities, ['MgO'])
    #return np.array([one_peak(peak) for peak in pattern.peaks.peaks])

In [7]:
signal_to_background(ds)

array([ 0.13553059,  1.11799593,  2.3348276 ])

In [28]:
signal_to_background(ds)

array([ 0.13553059,  1.11799593,  2.3348276 ])

In [5]:
#corr = cm_corrections(ds)

In [6]:
ratios = signal_to_background(ds)

In [7]:
ratios

array([ 0.13553059,  1.11799593,  2.3348276 ])

In [8]:
#[plt.plot(f(range(700)), label = '') for f in cm_corrections(ds)]
[plt.plot(f(range(700)), label = '') for f in interpolated_cms(ds)]
#[plt.plot(f(range(700)), label = '') for f in signal_to_background(ds)]
plt.show()

In [10]:
from dataccess import nbfunctions

In [5]:
nominal_energy = 8965.
from dataccess import api
@api.register_input_detids('quad2', 'si')
def powder_pattern_evt_2(quad2 = None, si = None, **kwargs):
    from dataccess import xrd
    import numpy as np
    from dataccess.peakfinder import peakfilter_frame
    from dataccess import mec
    cleaned_quad2 = peakfilter_frame(quad2, detid = 'quad2')
    pat = xrd.Pattern.from_dataset(cleaned_quad2, 'quad2', ['MgO'], label  = 'test')
    cm_si = mec.si_imarr_cm_3(si)
    deltaE = cm_si - nominal_energy
    theta_rad = np.deg2rad(pat.angles)
    theta_correction = np.rad2deg(2 * np.tan(theta_rad/2) * deltaE/nominal_energy)
    rescale = (pat.angles + theta_correction)/pat.angles
    angles, intensities = utils.regrid(pat.angles, pat.intensities, rescale)
    return np.array([angles, intensities])

In [6]:
def peak_angles(dataresult, compound_list = ['MgO']):
    """
    dataresult : DataResult instance with each value in dataresult.event_data a spectrum of the form [x, y]
    """
    mean_angles, mean_intensities = dataresult.mean
    def peak_nominal_index(peak):
        indices = peak.crop_indices(mean_angles, peak.peak_width)
        return indices[len(indices)/2]
    angles, intensities = dataresult
    pattern = xrd.Pattern(angles, intensities, compound_list = compound_list)
    nominal_peak_indices = [peak_nominal_index(peak) for peak in pattern.peaks.peaks]
    indices = [nearest_max(mean_intensities, i) for i in nominal_peak_indices]
    return [mean_angles[i] for i in indices]

def peak_cms(dataresult, compound_list = ['MgO']):
    from copy import deepcopy
    dataresult_copy = deepcopy(dataresult)
    angles, intensities = dataresult_copy
    pattern = xrd.Pattern(angles, intensities, compound_list = compound_list)
    pattern_angles, pattern_intensities = dataresult_copy.mean
    angles = peak_angles(dataresult_copy, compound_list = compound_list)
    for angle, peak in zip(angles, pattern.peaks.peaks):
        peak.angle = angle
    #pdb.set_trace()
    return [peak.center_of_mass(pattern_angles, pattern_intensities) for peak in pattern.peaks.peaks]

#def make_shifter_frame_processor(dataset, pattern_shifter_func):
#    shifter = make_shifter_function(dataset, pattern_shifter_func)
#    from dataccess import api
#    #peak_angles = peak_cms(uncorrected, compound_list = ['MgO'])
#    @api.register_input_detids('quad2', 'si')
#    def frame_processor(quad2 = None, si = None, **kwargs):
#        from dataccess.peakfinder import peakfilter_frame
#        # TODO set peak angles
#        
#        pat = xrd.Pattern.from_dataset(peakfilter_frame(quad2, detid = 'quad2'), 'quad2', ['MgO'], label  = 'test')#, peak_angles = peak_angles)
#        return shifter(quad2 = quad2, si = si, **kwargs)
#    return frame_processor

In [7]:
uncorrected = ds.evaluate('quad2', frame_processor = powder_pattern_single_frame,
                               event_data_getter = utils.identity)
angles, intensities = uncorrected.mean
pat = xrd.Pattern(angles, intensities, compound_list = ['MgO'])
print pat.centers_of_mass()


[33.474475343634552, 38.887067273503995, 55.846854729200146]


In [8]:
from dataccess.mec import si_imarr_cm_3
energies = ds.evaluate('si', frame_processor = si_imarr_cm_3, event_data_getter = utils.identity)

In [9]:
smoothed_cm_interpolations = interpolated_cms(ds)
xmin, xmax = min(smoothed_cm_interpolations[0].x), max(smoothed_cm_interpolations[0].x)
event_numbers = range(xmin, xmax)
from dataccess.mec import si_imarr_cm_3
energies = ds.evaluate('si', frame_processor = si_imarr_cm_3, event_data_getter = utils.identity)

selected = smoothed_cm_interpolations[1]
plt.scatter(selected.x, selected.y, label = 'raw')
plt.plot(selected.x, selected(selected.x), label  ='interpolation, smoothed with sigma = 20')
plt.show()

In [10]:
# Check if variation in signal strength could be throwing off the unweighted mean CM value. Seems not, at least for
# this dataset.
integrals = signal_plus_background_trace(ds)
plt.plot(integrals[:, 1], label = '')
plt.show()

NameError: name 'signal_plus_background_trace' is not defined

> <ipython-input-10-468d15291210>(3)<module>()
      1 # Check if variation in signal strength could be throwing off the unweighted mean CM value. Seems not, at least for
      2 # this dataset.
----> 3 integrals = signal_plus_background_trace(ds)
      4 plt.plot(integrals[:, 1], label = '')
      5 plt.show()

ipdb> c


In [11]:
corrected_result = ds.evaluate('quad2', frame_processor = powder_pattern_evt_2, event_data_getter = utils.identity)
uncorrected_result = ds.evaluate('quad2', frame_processor = powder_pattern_single_frame,
                        event_data_getter = utils.identity)

In [16]:
plt.plot(np.mean(np.array(list(uncorrected_result.iter_event_value_pairs()))[:, 1], axis = 0), label = '') # not corrected
plt.plot(np.mean(np.array(list(corrected_result.iter_event_value_pairs()))[:, 1], axis = 0), label = '') # corrected

In [17]:
plt.show()

In [12]:
def wobble_shifter(pattern, si = None, nevent = None, shift_scale = -1., corr = None, **kwargs):
    angles, intensities = pattern.shift_peaks(corr(nevent), shift_scale = shift_scale)
    return angles, intensities

In [6]:
#wobbleshift = make_shifter_frame_processor(ds, wobble_shifter)

In [13]:
wobbleshift = make_shifter_function(ds, wobble_shifter)

In [17]:
range(-10, 10)

[-10, -9, -8, -7, -6, -5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [6]:
wobbles=[]
for s in range(-5, 5):
    wobbleshift = make_shifter_function(ds, wobble_shifter, shift_scale = s)
    @api.register_input_detids('quad2', 'si')
    def wobbleshift2(quad2 = None, si = None, **kwargs):
        return wobbleshift(quad2 = quad2, si = si, **kwargs)
    wobbles.append(ds.evaluate('quad2', frame_processor = wobbleshift2, event_data_getter = utils.identity))

In [14]:
wobbles = []

In [15]:
wobbleshift = make_shifter_function(ds, wobble_shifter, shift_scale = 11.)
@api.register_input_detids('quad2', 'si')
def wobbleshift2(quad2 = None, si = None, **kwargs):
    return wobbleshift(quad2 = quad2, si = si, **kwargs)
wobbles.append(ds.evaluate('quad2', frame_processor = wobbleshift2, event_data_getter = utils.identity))

IOError: could not get source code

> /reg/neh/home/ohoidn/venv_test/mecana2/lib/python2.7/site-packages/dill/source.py(144)findsource()
    142 
    143     if not lines:
--> 144         raise IOError('could not get source code')
    145 
    146     #FIXME: all below may fail if exec used (i.e. exec('f = lambda x:x') )

ipdb> up
> /reg/neh/home/ohoidn/venv_test/mecana2/lib/python2.7/site-packages/dill/source.py(240)getblocks()
    238     DEPRECATED: use 'getsourcelines' instead
    239     """
--> 240     lines, lnum = findsource(object)
    241 
    242     if ismodule(object):

ipdb> p getsourcelines
<function getsourcelines at 0x7f5f9e5f3e60>
ipdb> p object
<function wobbleshift2 at 0x7f5f6f69f500>
ipdb> p getsourcelines(object)
*** IOError: IOError('could not get source code',)
ipdb> up
> /reg/neh/home/ohoidn/venv_test/mecana2/lib/python2.7/site-packages/dill/source.py(314)getsourcelines()
    312     If lstrip=True, ensure there is no indentation in the first line of code.
    313     If enclosing=True, then also

In [12]:
wobbleshift = make_shifter_function(ds, wobble_shifter, shift_scale = 10.)

In [13]:
@api.register_input_detids('quad2', 'si')
def wobbleshift2(quad2 = None, si = None, **kwargs):
    return wobbleshift(quad2 = quad2, si = si, **kwargs)

In [12]:
c = cm_corrections(ds)

In [14]:
def cm_corrections(ds, fudge_factor = 1.):
    """
    fudge_factor is a scaling parameter for the signal/background ratio.
    """
    interpolators = interpolated_cms(ds)
    ratios = signal_to_background(ds)
    return lambda nevent: None
    def newfunc(nevent):
        return np.array([(1 + fudge_factor/ratios[i]) * interpolators[i](nevent)[0] for i in range(len(interpolators))])
    return newfunc

In [14]:
#wobble_corrected = ds.evaluate('quad2', frame_processor = make_shifter_frame_processor(ds, wobble_shifter), event_data_getter = utils.identity, corr = cm_corrections(ds))
#, corr = cm_corrections(ds)
wobble_corrected = ds.evaluate('quad2', frame_processor = wobbleshift2, event_data_getter = utils.identity)

In [19]:
bothshift = make_shifter_frame_processor(ds, both_shifter)
both_corrected = ds.evaluate('quad2', frame_processor = bothshift,
                        event_data_getter = utils.identity)

In [14]:
#plt.plot(np.mean(np.array(list(uncorrected_result.iter_event_value_pairs()))[:, 1], axis = 0), label = '') # not corrected
#plt.plot(np.mean(np.array(list(corrected_result.iter_event_value_pairs()))[:, 1], axis = 0), label = '') # corrected
[plt.plot(np.mean(np.array(list(w.iter_event_value_pairs()))[:, 1], axis = 0), label = '') for w in wobbles 
]
plt.show()

In [16]:
plt.plot(np.mean(np.array(list(uncorrected_result.iter_event_value_pairs()))[:, 1], axis = 0), label = '') # not corrected
plt.plot(np.mean(np.array(list(corrected_result.iter_event_value_pairs()))[:, 1], axis = 0), label = '') # corrected
plt.plot(np.mean(np.array(list(wobble_corrected.iter_event_value_pairs()))[:, 1], axis = 0), label = '') # corrected
#plt.plot(np.mean(np.array(list(both_corrected.iter_event_value_pairs()))[:, 1], axis = 0), label = '') # corrected
plt.show()

In [25]:
plt.plot(np.mean(np.array(list(uncorrected_result.iter_event_value_pairs()))[:, 1], axis = 0), label = '') # not corrected
plt.plot(np.mean(np.array(list(corrected_result.iter_event_value_pairs()))[:, 1], axis = 0), label = '') # corrected
plt.plot(np.mean(np.array(list(wobble_corrected.iter_event_value_pairs()))[:, 1], axis = 0), label = '') # corrected
#plt.plot(np.mean(np.array(list(both_corrected.iter_event_value_pairs()))[:, 1], axis = 0), label = '') # corrected
plt.show()

In [20]:
plt.plot(np.mean(np.array(list(uncorrected_result.iter_event_value_pairs()))[:, 1], axis = 0), label = '') # not corrected
plt.plot(np.mean(np.array(list(corrected_result.iter_event_value_pairs()))[:, 1], axis = 0), label = '') # corrected
plt.plot(np.mean(np.array(list(wobble_corrected.iter_event_value_pairs()))[:, 1], axis = 0), label = '') # corrected
#plt.plot(np.mean(np.array(list(both_corrected.iter_event_value_pairs()))[:, 1], axis = 0), label = '') # corrected
plt.show()

In [15]:
plt.plot(np.mean(np.array(list(uncorrected_result.iter_event_value_pairs()))[:, 1], axis = 0), label = '') # not corrected
plt.plot(np.mean(np.array(list(corrected_result.iter_event_value_pairs()))[:, 1], axis = 0), label = '') # corrected
plt.plot(np.mean(np.array(list(wobble_corrected.iter_event_value_pairs()))[:, 1], axis = 0), label = '') # corrected
#plt.plot(np.mean(np.array(list(both_corrected.iter_event_value_pairs()))[:, 1], axis = 0), label = '') # corrected
plt.show()

In [13]:
plt.plot(np.mean(np.array(list(uncorrected_result.iter_event_value_pairs()))[:, 1], axis = 0), label = '') # not corrected
plt.plot(np.mean(np.array(list(corrected_result.iter_event_value_pairs()))[:, 1], axis = 0), label = '') # corrected
plt.plot(np.mean(np.array(list(wobble_corrected.iter_event_value_pairs()))[:, 1], axis = 0), label = '') # corrected
#plt.plot(np.mean(np.array(list(both_corrected.iter_event_value_pairs()))[:, 1], axis = 0), label = '') # corrected
plt.show()

In [16]:
plt.plot(np.mean(np.array(list(uncorrected_result.iter_event_value_pairs()))[:, 1], axis = 0), label = '') # not corrected
plt.plot(np.mean(np.array(list(corrected_result.iter_event_value_pairs()))[:, 1], axis = 0), label = '') # corrected
plt.plot(np.mean(np.array(list(wobble_corrected.iter_event_value_pairs()))[:, 1], axis = 0), label = '') # corrected
#plt.plot(np.mean(np.array(list(both_corrected.iter_event_value_pairs()))[:, 1], axis = 0), label = '') # corrected
plt.show()

In [20]:
plt.plot(np.mean(np.array(list(uncorrected_result.iter_event_value_pairs()))[:, 1], axis = 0), label = '') # not corrected
plt.plot(np.mean(np.array(list(corrected_result.iter_event_value_pairs()))[:, 1], axis = 0), label = '') # corrected
plt.plot(np.mean(np.array(list(wobble_corrected.iter_event_value_pairs()))[:, 1], axis = 0), label = '') # corrected
#plt.plot(np.mean(np.array(list(both_corrected.iter_event_value_pairs()))[:, 1], axis = 0), label = '') # corrected
plt.show()

In [20]:
plt.plot(np.mean(np.array(list(uncorrected_result.iter_event_value_pairs()))[:, 1], axis = 0), label = '') # not corrected
plt.plot(np.mean(np.array(list(corrected_result.iter_event_value_pairs()))[:, 1], axis = 0), label = '') # corrected
plt.plot(np.mean(np.array(list(wobble_corrected.iter_event_value_pairs()))[:, 1], axis = 0), label = '') # corrected
#plt.plot(np.mean(np.array(list(both_corrected.iter_event_value_pairs()))[:, 1], axis = 0), label = '') # corrected
plt.show()

In [19]:
plt.plot(np.mean(np.array(list(uncorrected_result.iter_event_value_pairs()))[:, 1], axis = 0), label = '') # not corrected
plt.plot(np.mean(np.array(list(corrected_result.iter_event_value_pairs()))[:, 1], axis = 0), label = '') # corrected
plt.plot(np.mean(np.array(list(wobble_corrected.iter_event_value_pairs()))[:, 1], axis = 0), label = '') # corrected
plt.show()

In [22]:
plt.plot(np.mean(np.array(list(uncorrected_result.iter_event_value_pairs()))[:, 1], axis = 0), label = '') # not corrected
plt.plot(np.mean(np.array(list(corrected_result.iter_event_value_pairs()))[:, 1], axis = 0), label = '') # corrected
plt.plot(np.mean(np.array(list(wobble_corrected.iter_event_value_pairs()))[:, 1], axis = 0), label = '') # corrected

In [23]:
plt.show()

In [164]:
[plt.plot(f(range(700)), label = '') for f in cm_corrections(ds)]
plt.show()

In [130]:
[plt.plot(f(range(700)), label = '') for f in interpolated_cms(ds)]
plt.show()

In [121]:
[plt.plot(f(range(700)), label = '') for f in interpolated_cms(ds, bgsub = True)]
plt.show()

In [27]:
[plt.plot(t, label = '') for t in icm_smooth(range(700)).T]
plt.show()

In [8]:
import mpi4py

In [61]:
interp, trace = correct_cms[2:]

NameError: name 'correct_cms' is not defined

> <ipython-input-61-79488de5474b>(1)<module>()
----> 1 interp, trace = correct_cms[2:]

ipdb> c


In [18]:
trace.mean

array([ 33.39296079,  38.87573271,  55.89671273])

In [13]:
interp(np.arange(10))

array([[ 0.12642347,  0.08372576, -0.09902238],
       [ 0.1262813 ,  0.08389837, -0.09873308],
       [ 0.12600547,  0.08424265, -0.09815446],
       [ 0.12558645,  0.08475663, -0.0972889 ],
       [ 0.125026  ,  0.0854407 , -0.09613842],
       [ 0.12432897,  0.08628378, -0.09470953],
       [ 0.12349874,  0.08729046, -0.09300988],
       [ 0.12253972,  0.08845454, -0.09104228],
       [ 0.12145323,  0.08976999, -0.08881661],
       [ 0.12024308,  0.09123016, -0.08633625]])

In [40]:
cms_nosub, smoothed_cms_nosub, smoothed_cms_nosub_interp = cm_variation_scatter(ds, bgsub=False)

In [94]:
def smooth_cm_200_interp_nosub(n_evt):
    cms_nosub, smoothed_cms_nosub, smoothed_cms_nosub_interp = cm_variation_scatter(ds, bgsub=False)
    sampled = smoothed_cms_nosub_interp(n_evt)[:, 1]
    if hasattr(n_evt, '__iter__'):
        return sampled
    else:
        return sampled[0]
    
def smooth_cm_200_interp(n_evt):
    cms, smoothed_cms, smoothed_cms_interp = cm_variation_scatter(ds, bgsub=True)
    sampled = smoothed_cms_interp(n_evt)[:, 1]
    if hasattr(n_evt, '__iter__'):
        return sampled
    else:
        return sampled[0]

In [95]:
print smooth_cm_200_interp(100)
print smooth_cm_200_interp_nosub(100)

0.117893981223
0.0473596990865


In [48]:
smoothed_cm_interp(10)

array([[ 33.51896685,  38.97173591,  55.81492828]])

In [42]:
plt.plot(smoothed_cm_interp(np.arange(600))[:, 1])
plt.show()

In [37]:
smoothed_cms

array([[ 33.49050786,  38.93583643,  56.15932785],
       [ 33.49049268,  38.93585015,  56.15968032],
       [ 33.49046268,  38.9358774 ,  56.16039872],
       ..., 
       [ 33.4726454 ,  38.89576205,  55.50960739],
       [ 33.47263536,  38.89578173,  55.50973153],
       [ 33.47263042,  38.89579134,  55.50980709]])

In [15]:
a = np.array(range(10))

In [16]:
a < 1

array([ True, False, False, False, False, False, False, False, False, False], dtype=bool)

In [27]:
def plt_smoothed(i, dataset, sigma_max = 0, mask = None):
    dataset = dataset[:, i]
    if sigma_max > 0 and mask is None:
        mask = np.abs(dataset - np.mean(dataset)) < sigma_max * np.std(dataset)
    if mask is not None:
        dataset = dataset[mask]
    plt.plot(dataset/np.mean(dataset), label  =str (i))
    return mask

In [14]:
[plt_smoothed(i, smoothed_cms) for i in range(3)]
plt.show()

In [31]:
masks = [plt_smoothed(i, cms, sigma_max= 2) for i in range(3)]
#plt.show()
#[plt_smoothed(i, smoothed_cms_nosub) for i in range(3)]
plt.show()

In [ ]:
masks

In [36]:
masks = [plt_smoothed(i, smoothed_cms, sigma_max= 0, mask = m) for i, m in enumerate(masks)]
masks = [plt_smoothed(i, smoothed_cms, sigma_max= 0) for i, m in enumerate(masks)]
#plt.show()
#[plt_smoothed(i, smoothed_cms_nosub) for i in range(3)]
plt.show()

In [77]:
[plt_smoothed(i, smoothed_cms) for i in range(3)]
#plt.show()
[plt_smoothed(i, smoothed_cms_nosub) for i in range(3)]
plt.show()

In [23]:
plt.scatter(cms[:, 0]/np.mean(cms[:, 0]))
plt.scatter(cms[:, 1]/np.mean(cms[:, 1]))
plt.scatter(cms[:, 2]/np.mean(cms[:, 2]))
plt.show()

In [9]:
cm_variation_scatter_plot(1)
#cm_variation_scatter_plot(2)

In [17]:
x = plot_xrd(datasets[:1], compound_list = ['MgO'], normalization = 'integral_bgsubbed',
         plot_patterns = True,  show_image = False, detectors = ['quad2', 'quad1'],
         frame_processor = peakfilter_frame)

In [18]:
pks = list(x.iter_peaks())

In [19]:
pk = pks[1]

In [20]:
pat = x.patterns[0]

In [21]:
fit = pat.fit_backgrounds()[1]

In [22]:
plt.plot(fit.xfit, fit.yfit)

In [23]:
plt.show()

In [27]:
x = plot_xrd(datasets[:1], compound_list = ['MgO'], normalization = None,
         plot_patterns = True,  show_image = False, detectors = ['quad2', 'quad1'],
         frame_processor = peakfilter_frame)

In [40]:
x2 = eval_xrd(datasets[:1], ['MgO'], normalization = 'integral_bgsubbed', detectors = ['quad2', 'quad1'],
              frame_processor = peakfilter_frame)

In [47]:
pat = x2.patterns[0]
fit = pat.fit_backgrounds()[1]

In [42]:
plt.plot(fit.xfit, fit.yfit)

In [43]:
plt.show()

In [14]:
def ith_peak_cm_bgsubbed(i, bg_pat):
    def powder_pattern_cm(imarr = None, **kwargs):
        import numpy as np
        from dataccess.peakfinder import peakfilter_frame
        from dataccess import xrd
        dss = xrd.Pattern.from_dataset(peakfilter_frame(imarr, detid = 'quad2'), 'quad2', ['MgO'], label  = 'test')
        
        return dss.centers_of_mass(bg_pattern = bg_pat)[i]
    return powder_pattern_cm

In [15]:
xrd

<dataccess.xrd.XRD instance at 0x7fe05bb4b128>

In [13]:
from dataccess.lk20 import ith_peak_cm_bgsubbed

In [14]:
ith_peak_cm_bgsubbed(1, 1)

<function dataccess.lk20.powder_pattern_cm>

In [5]:
def cm_variation_scatter_plot_bgsubbed(peak_index):
    from dataccess.lk20 import ith_peak_cm_bgsubbed
    x = eval_xrd([ds], ['MgO'], normalization = 'integral_bgsubbed', detectors = ['quad2', 'quad1'],
              frame_processor = peakfilter_frame)
    bg_pat = x.patterns[0]
    trace = ds.evaluate('quad2', frame_processor = ith_peak_cm_bgsubbed(peak_index, bg_pat), event_data_getter = utils.identity)
    event_numbers, cms = range(len(trace.flat_event_data())), trace.flat_event_data()
    smoothed_cm_interp = utils.extrap1d(interp1d(event_numbers, gfilt(cms, 20)))
    smoothed_cms = smoothed_cm_interp(event_numbers)
    cms_highpass = cms - smoothed_cms
    #from dataccess.mec import si_imarr_cm_3
    #energies = ds.evaluate('si', frame_processor = si_imarr_cm_3, event_data_getter = utils.identity)

    plt.scatter(event_numbers, cms, label = 'raw')
    plt.plot(np.array(event_numbers), smoothed_cms, label  ='interpolation, smoothed with sigma = 20')
    plt.show()

In [17]:
def g():
    def f():
        return f.y
    f.y = 2
    return f

In [18]:
f2 =  g()

In [19]:
f2()

2

In [20]:
def ith_peak_cms_bgsubbed(bgpat2 = None, **kwargs):
    def foobar(imarr = None, **kwargs):
        from dataccess import xrd
        from dataccess.peakfinder import peakfilter_frame
        a = foobar.bgpat2
        #dss2 = xrd.Pattern.from_dataset(peakfilter_frame(imarr,detid = 'quad2'),'quad2',['MgO'],label= 'test')
        #b = bgpat2#bgpat2#.centers_of_mass# ( bgpat2 )
        #(bg_pat2)#[0]
        return 1.
    foobar.bgpat2 = bgpat2
    return foobar

In [7]:
def cm_variation_scatter_plot_bgsubbed():
    x = eval_xrd([ds], ['MgO'], normalization = 'integral_bgsubbed', detectors = ['quad2', 'quad1'], frame_processor = peakfilter_frame)
    bg_pat = x.patterns[0]
#        def foobar(imarr = None, **kwargs):
#            dss = xrd.Pattern.from_dataset(
#                peakfilter_frame(imarr, detid = 'quad2'),
#                'quad2', ['MgO'], label  = 'test')
#            return np.array(dss.centers_of_mass(bg_pattern = bg_pat))
    trace = ds.evaluate('quad2', frame_processor = ith_peak_cms_bgsubbed(bg_pat), event_data_getter = utils.identity)
    event_numbers, cms = range(len(trace.flat_event_data())), trace.flat_event_data()
    smoothed_cm_interp = utils.extrap1d(interp1d(event_numbers, gfilt(cms, 20)))
    smoothed_cms = smoothed_cm_interp(event_numbers)
    cms_highpass = cms - smoothed_cms
    #from dataccess.mec import si_imarr_cm_3
    #energies = ds.evaluate('si', frame_processor = si_imarr_cm_3, event_data_getter = utils.identity)

    plt.scatter(event_numbers, cms, label = 'raw')
    plt.plot(np.array(event_numbers), smoothed_cms, label  ='interpolation, smoothed with sigma = 20')
    plt.show()

In [27]:
from dataccess import lk20

In [25]:
f = lk20.ith_peak_cms_bgsubbed(1)

In [5]:
#from dataccess.lk20 import cvsp0
from dataccess.lk20 import cm_variation_scatter_plot_bgsubbed

In [6]:
cm_variation_scatter_plot_bgsubbed(0, ds)

IndentationError: unexpected indent (<unknown>, line 1)

In [10]:
%pdb

Automatic pdb calling has been turned ON


In [72]:
def interpolated_subtraction(x1, y1, x2, y2):
    """
    For functions f, g sampled by f(x1) = y1 and g(x2) = y2, where x1 and x2 are 1d arrays, return h(x1) = f(x1) - g(x1),
    using interpolation if the provided domains x1 and x2 do not match
    """
    from scipy.interpolate.interpolate import interp1d
    from dataccess import utils
    reload(utils)
    extrap1 = utils.extrap1d(interp1d(x1, y1), default = 0.)
    extrap2 = utils.extrap1d(interp1d(x2, y2), default = 0.)
    return extrap1(x1) - extrap2(x1)


In [76]:
plt.plot(pat.angles, interpolated_subtraction(pat.angles, pat.intensities, fit.xfit, fit.yfit), label = '')

In [77]:
plt.show()

In [63]:
plt.plot(pat.angles, utils.extrap1d(interp1d(fit.xfit, fit.yfit), default = 0.)(pat.angles))
plt.show()

In [59]:
type(interp1d(fit.xfit, fit.yfit)) == scipy.interpolate.interpolate.interp1d

True

In [58]:
import scipy

In [60]:
reload(utils)

<module 'dataccess.utils' from '/reg/neh/home5/ohoidn/venv_test/mecana2/lib/python2.7/site-packages/dataccess-1.0-py2.7.egg/dataccess/utils.pyc'>

In [18]:
def get_ast(obj):
    source = inspect.getsource(obj)
    tree = ast.parse(source)
    return astor.dump(tree)

In [9]:
import inspect

In [10]:
import ast

In [11]:
import astor

In [12]:
from dataccess.lk20 import ith_peak_cm_bgsubbed

In [19]:
get_ast(ith_peak_cm_bgsubbed)

"Module(\n    body=[\n        FunctionDef(name='ith_peak_cm_bgsubbed',\n            args=arguments(args=[Name(id='i'), Name(id='bg_pat')], vararg=None, kwarg=None, defaults=[]),\n            body=[\n                FunctionDef(name='powder_pattern_cm',\n                    args=arguments(args=[Name(id='imarr')], vararg=None, kwarg='kwargs', defaults=[Name(id='None')]),\n                    body=[\n                        ImportFrom(module='dataccess.peakfinder',\n                            names=[alias(name='peakfilter_frame', asname=None)],\n                            level=0),\n                        ImportFrom(module='dataccess', names=[alias(name='xrd', asname=None)], level=0),\n                        Assign(targets=[Name(id='dss')],\n                            value=Call(\n                                func=Attribute(value=Attribute(value=Name(id='xrd'), attr='Pattern'),\n                                    attr='from_dataset'),\n                                args=[\n    

In [15]:
callable(ith_peak_cm_bgsubbed)

True